# Pipelines

Pipelines simplifiy our workflow by chaining several steps into one process.

"Pipelines can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification." [Source](https://scikit-learn.org/stable/modules/compose.html)

_Datasets: Diabetes, Gapminder._

**Outline:**

* Pipeline for Classification
  - Imputation in a Pipeline
  - Centering and Scaling in a Pipeline
  - Pipeline and Hyperparameter Tuning
* Pipeline for Regression

In [46]:
# Import necessary modules
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

## Pipeline for Classification

In [47]:
df = pd.read_csv("datasets/diabetes.csv")

# Extract features and target from the df
X = df.drop('Outcome', axis = 1).values # features
y = df['Outcome'].values # target

# Create training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

### Imputation in a Pipeline

In [48]:
# Import the Imputer module
#from sklearn.preprocessing import Imputer > Imputer was deprecated
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# Setup the Imputation transformer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Instantiate the SVC classifier
clf = SVC()

# Setup the pipeline steps
steps = [('imputation', imp),
        ('SVM', clf)]

# # Setup the pipeline steps
# steps = [('imputation', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
#         ('SVM', SVC())]

# Create the pipeline
pipeline = Pipeline(steps)

# Fit the pipeline to the train set
pipeline.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = pipeline.predict(X_test)

# Compute metrics
print(classification_report(y_test, y_pred))

print(pipeline.score(X_test, y_test))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80       206
           1       0.00      0.00      0.00       102

   micro avg       0.67      0.67      0.67       308
   macro avg       0.33      0.50      0.40       308
weighted avg       0.45      0.67      0.54       308

0.6688311688311688


### Centering and Scaling in a Pipeline

In [49]:
# Import the necessary modules
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# Setup the pipeline steps
steps = [('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())]
        
# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# Create train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline to the training set
knn_scaled = pipeline.fit(X_train, y_train)

# Instantiate and fit a k-NN classifier to the unscaled data
knn_unscaled = KNeighborsClassifier().fit(X_train, y_train)

# Compute and print metrics
print('Accuracy with Scaling: {}'.format(knn_scaled.score(X_test, y_test)))
print('Accuracy without Scaling: {}'.format(knn_unscaled.score(X_test, y_test)))


Accuracy with Scaling: 0.7077922077922078
Accuracy without Scaling: 0.7077922077922078


### Pipeline and Hyperparameter Tuning

In [50]:
from sklearn.model_selection import GridSearchCV

# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('SVM', SVC())]

pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'SVM__C':[1, 10, 100],
              'SVM__gamma':[0.1, 0.01]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=21)

# Instantiate the GridSearchCV object: cv
cv = GridSearchCV(pipeline, parameters, cv=5)

# Fit to the training set
cv.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Compute and print metrics
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))


Accuracy: 0.7532467532467533
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        94
           1       0.78      0.52      0.62        60

   micro avg       0.75      0.75      0.75       154
   macro avg       0.76      0.71      0.72       154
weighted avg       0.76      0.75      0.74       154

Tuned Model Parameters: {'SVM__C': 1, 'SVM__gamma': 0.01}


## Pipeline for Regression

In [51]:
# Read the gapminder file into df
df = pd.read_csv("datasets/gapminder.csv")

# Create dummy variables with dropping the first
df = pd.get_dummies(df, drop_first=True)

# Create the feature array
X = df.drop(['life'], axis = 1).values # features (columns other than 'life')

# Create the target array
y = df['life'].values # target

In [52]:
import numpy as np
from sklearn.linear_model import ElasticNet

# Setup the pipeline steps: steps
steps = [('imputation', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
         ('scaler', StandardScaler()),
         ('elasticnet', ElasticNet())]

# Create the pipeline: pipeline 
pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'elasticnet__l1_ratio':np.linspace(0,1,20)}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create the GridSearchCV object: gm_cv
gm_cv = GridSearchCV(pipeline, parameters, cv=5)

# Fit to the training set
gm_cv.fit(X_train, y_train)

# Compute and print the metrics
r2 = gm_cv.score(X_test, y_test)
print("Tuned ElasticNet Alpha: {}".format(gm_cv.best_params_))
print("Tuned ElasticNet R squared: {}".format(r2))

Tuned ElasticNet Alpha: {'elasticnet__l1_ratio': 1.0}
Tuned ElasticNet R squared: 0.8862016549771036
